In [ ]:
import numpy as np
from numba import jit

import matplotlib.pyplot as plt

from functools import partial

import sys
sys.path.append('../miniMD')
from miniMD import *

# Umbrella Sampling

Umbrella sampling is a widespread method to obtain free energies as a function of an arbitrary coordinate $\zeta$. Here, an umbrella function should guide the sampling towards the regions of interest. While this guiding function can be of any form, in practice we often employ a simple harmonic potential as a function of $\zeta$:

$$U_B(x) = U(x) + U_{bias}(\zeta)$$
$$U_B(x) = U(x) + \frac{k}{2} (\zeta(x) - \hat\zeta )^2$$

where $\hat\zeta$ describes the bias position and $k$ is the force constant. To efficiently obtain a density along the whole coordinate, we can sample the denisties at different bias centers (windows) $\{\hat\zeta_0, \hat\zeta_1, ..., \hat\zeta_N\}$ and later reconstruct the full free energy. Each window then samples from the pdf:

$$p(x) = \frac{Z}{Z_i} p_i(\hat\zeta) \exp\left[\beta \frac{k}{2} (\zeta(x) - \hat\zeta_i )^2\right] $$

While $p_i(\hat\zeta) \exp\left[\beta \frac{k}{2} (\zeta(x) - \hat\zeta_i )^2\right]$ can be estimated based on the simulation, $Z_i$ is not easily obtainable. This factor corresponds to an offset of the free energies in each window with respect to each other and methods such as WHAM were developed to determine this offset self consistently.

Lets start as always with a energy and force function, in this case it is the same as in the TI/FEP notebook:

In [ ]:

@jit(nopython=True) 
def custom_potential_energy(current_x : np.ndarray) -> float:
    """
    Calculates the potential energy given a configuration current_x. 

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration to be propagated. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    U : float
        Potential energy of the configuration

    """

    return 10 * ((current_x[0]**2 - 1)**2 + (current_x[0] - current_x[1])**2)


@jit(nopython=True) 
def custom_force_function(current_x : np.ndarray) -> np.ndarray:
    """
    Calculates the force given a configuration current_x.

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration to be propagated. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    force : np.ndarray
        Force corresponding the provided configuration.

    """
    force = np.zeros(2)
    
    force[0] = -20 * (2 * current_x[0]**3 -current_x[0] - current_x[1])
    force[1] = 20 * (current_x[0] - current_x[1])

    return force


Next up, we need a definiton of our collective variable $\zeta$. For simplicity, we here use the $x^{(0)}$ coordinate of the system, although a better choice might be the diagonal $x-y$ (optional task):


In [ ]:
@jit(nopython=True) 
def identity(current_x : np.ndarray) -> float:

    return current_x

@jit(nopython=True) 
def custom_cv(current_x : np.ndarray) -> float:
    """
    Calculates the collective variable zeta given a configuration current_x.

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    zeta : float
        CV corresponding the provided configuration.
    """

    return current_x[..., 0]

The bias force can be added to the force arising from the systems potential energy landscape. To calculate the bias force, it is recommended to use the chain rule:

$$\frac{\partial}{\partial x} U_{bias}(\zeta(x)) = \frac{\partial}{\partial \zeta} U_{bias}(\zeta(x)) \times \frac{\partial}{\partial x} \zeta(x)$$

This ensures that you can easily switch the collective variable without having to rewrite the bias potential code and vice versa.

**1) Implement the following functions in the next code block:**

1) ''custom_cv_gradient'' should return $\frac{\partial}{\partial x} \zeta(x)$
1) ''bias_potential_energy'' should return $U_{bias}(\zeta(x))$ given a configuration x, a cv function, a bias center and a force constant
1) ''bias_force_function'' should return $\frac{\partial}{\partial x} U_{bias}(\zeta(x))$ accepting the above defined custom_cv_gradient function

In [ ]:

@jit(nopython=True) 
def bias_potential_energy(current_x : np.ndarray, cv_function : callable, bias_center : float, force_constant : float) -> np.ndarray:
    """
    Calculates the harmonic bias energy given a configuration current_x, a cv function and bias parameters. 

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration. The shape of the array(current_x.shape) can vary depending on the system which is simulated.
        
    cv_function : callable
        Function that accepts the current configuration and maps it onto the collective variable.

    bias_center : float
        Position of the bias (zeta hat).

    force_constant : float
        Force constant of the bias.

    Returns
    -------
    U_bias : float
        Bias energy corresponting to the configuration

    """
    return ...

@jit(nopython=True) 
def bias_force_function(current_x : np.ndarray, cv_function : callable, cv_gradient_function : callable, bias_center : float, force_constant : float) -> np.ndarray:
    """
    Calculates the force associated to a harmonic bias given a configuration current_x, a cv function, its gradient w.r.t current_x and bias parameters. 

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration. The shape of the array(current_x.shape) can vary depending on the system which is simulated.
        
    cv_function : callable
        Function that accepts the current configuration and maps it onto the collective variable.

    cv_gradient_function : callable
        Function that accepts the current configuration and returns the gradient w.r.t to the configuration.

    bias_center : float
        Position of the bias (zeta hat).

    force_constant : float
        Force constant of the bias.

    Returns
    -------
    F_bias : np.ndarray
        Force acting on x arising from the harmonic bias

    """

    return ...

@jit(nopython=True) 
def custom_cv_gradient(current_x : np.ndarray) -> np.ndarray:
    """
    Calculates the gradient of the cv w.r.t to the configuration.

    Parameters
    ----------
    current_x : np.ndarray
        Current configuration. The shape of the array(current_x.shape) can vary depending on the system which is simulated.

    Returns
    -------
    cv_gradient : np.ndarray
        Gradient of custom_cv w.r.t. current_x

    """
    
    return ...


Next up we can define the parameters for the simulation. Here only bias_centers and force_constant are new parameters. bias_centers contains a list of centers to sample and we use the same force constant in each window:

In [ ]:
equilibration_steps = 10000
n_steps_per_window = 50000
output_frequency = 1

beta = 1
timestep = 0.001
diffusion_coefficient = 1

bias_centers = np.linspace(-2,2,10)
force_constant = 100

initial_x = np.zeros(2)

assert equilibration_steps < n_steps_per_window, "Make sure you don't equilibrate longer than you simulate."
assert output_frequency < n_steps_per_window, "Make sure you don't output less often than you simulate."
assert output_frequency > 0, "The output frequency needs to be larger than 0"


For the main simulation loop, we need additionally to loop over the bias centers to perform a simulation in each window.

**2) Complete the simulation code to obtain a trajectory for each bias window**

In [ ]:
window_trajectories = []

for center in bias_centers:

    previous_x = initial_x.copy()
    
    total_frames = int(np.ceil((n_steps_per_window - equilibration_steps) / output_frequency))
    trajectory = np.zeros((total_frames , 2))

    for step in range(n_steps_per_window):

        current_force = ...
        
        previous_x = update_positions(previous_x, current_force, beta, timestep, diffusion_coefficient)

        if step >= equilibration_steps and step % output_frequency == 0:
            index = (step - equilibration_steps) // output_frequency
            trajectory[index] = previous_x

    window_trajectories.append(trajectory)

**3) Visualize the trajectories in each window using the plotting code below.**

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,4), dpi=180)


for i, t in enumerate(window_trajectories):

    ...



x_values = np.linspace(-2.6, 2.6, 100)
y_values = np.linspace(-2.6, 2.6, 100)

x_grid, y_grid = np.meshgrid(x_values, y_values)
energies = np.zeros((len(x_values), len(y_values)))

for i in range(len(x_values)):
    for j in range(len(y_values)):

        energies[i,j] = custom_potential_energy(np.array([x_grid[i, j], y_grid[i, j]]))


ax.contourf(x_grid, y_grid, energies, levels=np.linspace(0, 20, 10), cmap="RdBu_r")

ax.legend(frameon=False, loc="upper left", ncol=2, fontsize=6)
ax.set_xlabel("x$^{0}$")
ax.set_ylabel("x$^{1}$")

plt.show()

**4) Estimate the densities along $\zeta$ for each window (using np.histogram) and show the results in a single plot.**

In [ ]:
window_densities = []
bins = np.linspace(-2,2,100)
bin_centers = (bins[1:] + bins[:-1]) / 2

for i, t in enumerate(window_trajectories):

    ...

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

for i, d in enumerate(window_densities):

    ax.plot(bin_centers, d)
    ax.fill_between(bin_centers, np.zeros_like(d), d, alpha=0.5)
    
ax.set_xlabel("$\hat\zeta$")
ax.set_ylabel("$p_i(\hat\zeta)$")
plt.show()

**5) Estimate the free energies along $\zeta$ for each window (using np.histogram) and show the results in a single plot.**

In [ ]:
window_free_energies = []

for i, d in enumerate(window_densities):

    ...


In [ ]:
fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

for i, d in enumerate(window_free_energies):

    ax.plot(bin_centers, d)
    
ax.set_xlabel("$\hat\zeta$")
ax.set_ylabel("$F_i(\hat\zeta)$")
plt.show()

As a last step, we want to recover the unbiased distribution:

$$p(\hat\zeta) = \frac{Z}{Z_i} p_i(\hat\zeta) \exp\left[\beta \frac{k}{2} (\zeta(x) - \hat\zeta_i )^2\right] $$

which translates to the free energy:

$$F(\hat\zeta) = -\frac{1}{\beta} \left( \ln Z - \ln Z_i + \ln p_i(\hat\zeta) + \left[\beta \frac{k}{2} (\zeta(x) - \hat\zeta_i )^2\right] \right)$$
$$F(\hat\zeta) = F_i(\hat\zeta) - \left[\frac{k}{2} (\zeta(x) - \hat\zeta_i )^2\right] - \frac{1}{\beta} \left( \ln Z - \ln Z_i \right)$$
$$F(\hat\zeta) = F_i(\hat\zeta) - \left[\frac{k}{2} (\zeta(x) - \hat\zeta_i )^2\right] + \Delta F_i$$

The offset $\Delta F_i$ comes into play later since we now want to look at the free energy within a single window. You can see from the above equations that you have to subtract the bias potential from the free energy within a window to obtain the unbiased free energy $F(\hat\zeta)$ up to an additive constant. 

**6) Use the relation above to obtain the bias-corrected $F(\hat\zeta)$ for each window and plot the resulting free energies.**

In [ ]:
corrected_window_free_energies = []

for i, f in enumerate(window_free_energies):    

    ...


In [ ]:
fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

for i, d in enumerate(corrected_window_free_energies):

    ax.plot(bin_centers, d)
    
ax.set_xlabel(r"$\hat\zeta$")
ax.set_ylabel(r"$F_i(\hat\zeta)$")
plt.show()

Last but not least, we need to stich together the fragments obtained above. For this, we need to find the offset $\Delta F_i$ for each window, representing a shift up and down of each fragment above. In this example we can do this by hand. However, in practice, one can use methods such as WHAM to obtain a more accurate result.

**6) Fill in the offsets below one after the other by repeadetly plotting the reconstructed surface**

**7) Describe the origin of the pattern visible in the final offsets.**

In [ ]:

offset = np.zeros(len(corrected_window_free_energies))

offset[0] = 0
offset[1] = ...
offset[2] = ...
offset[3] = ...
offset[4] = ...
offset[5] = ...
offset[6] = ...
offset[7] = ...
offset[8] = ...
offset[9] = ...

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,3), dpi=180)

for i, d in enumerate(corrected_window_free_energies):

    ax.plot(bin_centers, d + offset[i])
    
ax.set_xlabel(r"$\hat\zeta$")
ax.set_ylabel(r"$F(\hat\zeta)$")
plt.show()